In [1]:
!pip install /kaggle/input/rsna-2022-whl/{pydicom-2.3.0-py3-none-any.whl,pylibjpeg-1.4.0-py3-none-any.whl,python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl}
!pip install -q /kaggle/input/nvidia-dali-nightly-cuda110-1230dev/nvidia_dali_nightly_cuda110-1.23.0.dev20230203-7187866-py3-none-manylinux2014_x86_64.whl
!pip install /kaggle/input/nvidia-dali-wheel/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl



use_compile = False
# if use_compile:
#upgrade pytorch to 1.12
!pip install /kaggle/input/pytorch112-cu113/{torch-1.12.1+cu113-cp37-cp37m-linux_x86_64.whl,torchvision-0.13.1+cu113-cp37-cp37m-linux_x86_64.whl}
!pip install /kaggle/input/torch-tensorrt-pkg/nvidia_pyindex-1.0.9-py3-none-any.whl
!mkdir -p /tmp/pip/cache/
!cp /kaggle/input/torch-tensorrt-pkg/nvidia-cublas-cu11-2022.4.8.xyz /tmp/pip/cache/nvidia-cublas-cu11-2022.4.8.tar.gz
!cp /kaggle/input/torch-tensorrt-pkg/nvidia-cuda-runtime-cu11-2022.4.25.xyz /tmp/pip/cache/nvidia-cuda-runtime-cu11-2022.4.25.tar.gz
!cp /kaggle/input/torch-tensorrt-pkg/nvidia-cudnn-cu11-2022.5.19.xyz /tmp/pip/cache/nvidia-cudnn-cu11-2022.5.19.tar.gz
!cp /kaggle/input/torch-tensorrt-pkg/nvidia_cublas_cu117-11.10.1.25-py3-none-manylinux1_x86_64.whl /tmp/pip/cache/
!cp /kaggle/input/torch-tensorrt-pkg/nvidia_cuda_runtime_cu117-11.7.60-py3-none-manylinux1_x86_64.whl /tmp/pip/cache/
!cp /kaggle/input/torch-tensorrt-pkg/nvidia_cudnn_cu116-8.4.0.27-py3-none-manylinux1_x86_64.whl /tmp/pip/cache/
!cp /kaggle/input/torch-tensorrt-pkg/nvidia_tensorrt-8.4.3.1-cp37-none-linux_x86_64.whl /tmp/pip/cache/
!pip install --no-index --find-links /tmp/pip/cache/ nvidia_tensorrt
#install torch_tensorrt
!pip install /kaggle/input/torch-tensorrt-pkg/torch_tensorrt-1.2.0-cp37-cp37m-linux_x86_64.whl

import os
os.environ['CUDA_MODULE_LOADING']='LAZY'
import torch_tensorrt
import tensorrt

Processing /kaggle/input/rsna-2022-whl/pydicom-2.3.0-py3-none-any.whl
Processing /kaggle/input/rsna-2022-whl/pylibjpeg-1.4.0-py3-none-any.whl
Processing /kaggle/input/rsna-2022-whl/python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
pydicom is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/nvidia-dali-wheel/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
Processing /kaggle/input/pytorch112-cu113/torch-1.12.1+cu113-cp37-cp37m-linux_x86_64.whl
Processing /kaggle/input/pytorch112-cu113/torchvision-0.13.1+cu113-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0
    Uninstalling torchvision-0.12.0:
      Successfully uninstalle

In [2]:
import torch.nn.functional as F
import nvidia.dali.fn as fn
import nvidia.dali.types as types
from nvidia.dali import pipeline_def
from nvidia.dali.types import DALIDataType
from pydicom.filebase import DicomBytesIO
from nvidia.dali.plugin.pytorch import to_torch_type

In [3]:
#we need to patch DALI for Int16 support


from nvidia.dali.backend import TensorGPU, TensorListGPU
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
from nvidia.dali import types
from nvidia.dali.plugin.base_iterator import _DaliBaseIterator
from nvidia.dali.plugin.base_iterator import LastBatchPolicy
import torch
import torch.utils.dlpack as torch_dlpack
import ctypes
import numpy as np
import torch.nn.functional as F
import pydicom

to_torch_type = {
    types.DALIDataType.FLOAT:   torch.float32,
    types.DALIDataType.FLOAT64: torch.float64,
    types.DALIDataType.FLOAT16: torch.float16,
    types.DALIDataType.UINT8:   torch.uint8,
    types.DALIDataType.INT8:    torch.int8,
    types.DALIDataType.UINT16:  torch.int16,
    types.DALIDataType.INT16:   torch.int16,
    types.DALIDataType.INT32:   torch.int32,
    types.DALIDataType.INT64:   torch.int64
}


def feed_ndarray(dali_tensor, arr, cuda_stream=None):
    """
    Copy contents of DALI tensor to PyTorch's Tensor.

    Parameters
    ----------
    `dali_tensor` : nvidia.dali.backend.TensorCPU or nvidia.dali.backend.TensorGPU
                    Tensor from which to copy
    `arr` : torch.Tensor
            Destination of the copy
    `cuda_stream` : torch.cuda.Stream, cudaStream_t or any value that can be cast to cudaStream_t.
                    CUDA stream to be used for the copy
                    (if not provided, an internal user stream will be selected)
                    In most cases, using pytorch's current stream is expected (for example,
                    if we are copying to a tensor allocated with torch.zeros(...))
    """
    dali_type = to_torch_type[dali_tensor.dtype]

    assert dali_type == arr.dtype, ("The element type of DALI Tensor/TensorList"
                                    " doesn't match the element type of the target PyTorch Tensor: "
                                    "{} vs {}".format(dali_type, arr.dtype))
    assert dali_tensor.shape() == list(arr.size()), \
        ("Shapes do not match: DALI tensor has size {0}, but PyTorch Tensor has size {1}".
            format(dali_tensor.shape(), list(arr.size())))
    cuda_stream = types._raw_cuda_stream(cuda_stream)

    # turn raw int to a c void pointer
    c_type_pointer = ctypes.c_void_p(arr.data_ptr())
    if isinstance(dali_tensor, (TensorGPU, TensorListGPU)):
        stream = None if cuda_stream is None else ctypes.c_void_p(cuda_stream)
        dali_tensor.copy_to_external(c_type_pointer, stream, non_blocking=True)
    else:
        dali_tensor.copy_to_external(c_type_pointer)
    return arr

In [4]:
# !pip install -q --no-index --find-links=/kaggle/input/timm-wheel timm pillow
!pip uninstall -y timm
import sys
sys.path.append('/kaggle/input/timm-0-6-12-tf-effv2s-1520-912')

In [5]:
import gdcm
import glob

import os
import pickle
import random
import math

from joblib import Parallel, delayed
import shutil
import cv2
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import timm
from albumentations  import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pydicom.pixel_data_handlers import apply_windowing
import dicomsdl
import gc

%matplotlib inline
import matplotlib.pyplot as plt

print('torch version:', torch.__version__)
print('timm version:', timm.__version__)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device:', device)

torch.backends.cuda.matmul.allow_tf32 = False

torch version: 1.12.1+cu113
timm version: 0.6.12
device: cuda


## Configuration

In [6]:
class config:
    seed = 10
    batch_size = 32
    n_folds = 5
    name = 'tf_efficientnetv2_s'
    SIZE = (1024, 1024)
class config2:
    seed = 10
    batch_size = 32
    n_folds = 4
    name = 'tf_efficientnetv2_s'
    SIZE = (912, 1520)
    
def seed_everything(seed: int):    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(config.seed)

# Load Data

In [7]:
test_df = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')
train_df = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')
train_df.drop(index=[16772, 16773, 16774, 16775, 16776, 53065], inplace=True)
display(test_df.head())
display(test_df.shape)

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id
0,2,10008,736471439,L,MLO,81,0,21,10008_L
1,2,10008,1591370361,L,CC,81,0,21,10008_L
2,2,10008,68070693,R,MLO,81,0,21,10008_R
3,2,10008,361203119,R,CC,81,0,21,10008_R


(4, 9)

In [8]:
windowing_stat_dict = dict()

## Utils

In [9]:
def convert_dicom_to_jpg(patient_id, img_id, save_folder=""):
    patient = patient_id
    image = img_id
    file = IMG_PATH + f"{patient}/{image}.dcm"
    dcmfile = pydicom.dcmread(file)

    if dcmfile.file_meta.TransferSyntaxUID == '1.2.840.10008.1.2.4.90':
        with open(file, 'rb') as fp:
            raw = DicomBytesIO(fp.read())
            ds = pydicom.dcmread(raw)
        offset = ds.PixelData.find(b"\x00\x00\x00\x0C")  #<---- the jpeg2000 header info we're looking for
        hackedbitstream = bytearray()
        hackedbitstream.extend(ds.PixelData[offset:])
        with open(save_folder + f"{patient}_{image}.jpg", "wb") as binary_file:
            binary_file.write(hackedbitstream)
            
    if dcmfile.file_meta.TransferSyntaxUID == '1.2.840.10008.1.2.4.70':
        with open(file, 'rb') as fp:
            raw = DicomBytesIO(fp.read())
            ds = pydicom.dcmread(raw)
        offset = ds.PixelData.find(b"\xff\xd8\xff\xe0")  #<---- the jpeg lossless header info we're looking for
        hackedbitstream = bytearray()
        hackedbitstream.extend(ds.PixelData[offset:])
        with open(save_folder + f"{patient}_{image}.jpg", "wb") as binary_file:
            binary_file.write(hackedbitstream)
        

@pipeline_def
def jpg_decode_pipeline(jpgfiles):
    jpegs, _ = fn.readers.file(files=jpgfiles)
    images = fn.experimental.decoders.image(jpegs, device='mixed', output_type=types.ANY_DATA, dtype=DALIDataType.UINT16)
    return images



# https://www.kaggle.com/code/hengck23/3hr-tensorrt-nextvit-example
def normalised_to_8bit(image, photometric_interpretation):
    if photometric_interpretation == 'MONOCHROME1':
        image = image.max()  - image
        rev = 1
    else:
        rev = 0
    xmin = image.min()
    xmax = image.max() 
    norm = np.empty_like(image, dtype=np.uint8)
    image = np.ascontiguousarray(image)
    dicomsdl.util.convert_to_uint8(image, norm, xmin, xmax)

    return norm, rev, xmin, xmax

# https://www.kaggle.com/code/hengck23/3hr-tensorrt-nextvit-example
def dicomsdl_to_numpy_image(ds, index=0):
    # https://stackoverflow.com/questions/44659924/returning-numpy-arrays-via-pybind11
    info = ds.getPixelDataInfo()

    shape = [info['Rows'], info['Cols']]
    dtype = info['dtype']
    outarr = np.empty(shape, dtype=dtype)
    ds.copyFrameData(index, outarr)
    return outarr


# https://www.kaggle.com/code/raddar/convert-dicom-to-np-array-the-correct-way/notebook
def save_array(patient_id, img_id, images_dict, voi_lut = False, fix_monochrome = True):
    path = IMG_PATH + f"{patient_id}/{img_id}.dcm"
    
    dicom = pydicom.dcmread(path)

    if f"{patient_id}/{img_id}.png" in images_dict:
        return None
    
    dicom = dicomsdl.open(path)      
    data = dicomsdl_to_numpy_image(dicom)
    data = data[5:-5, 5:-5]  
    rev_val = np.amax(data) # for sigmoid windowing
    data, rev, min_, max_ = normalised_to_8bit(data, dicom.getPixelDataInfo()['PhotometricInterpretation'])
    if type(dicom.WindowWidth) == list:
        center = dicom.WindowCenter[0]
        width = dicom.WindowWidth[0]
    else:
        center = dicom.WindowCenter
        width = dicom.WindowWidth
    y_range = 2**dicom.BitsStored - 1
    windowing_stat_dict[f'{img_id}'] = [min_, max_, center, width, y_range, rev, rev_val]
    
    data = cp.array(data)
    data = ExtractBreast_cupy(data)
    c_h = data.shape[0]
    if c_h > 2048:
        pad = c_h - 2048
        data2 = data[pad//2:-pad//2, :]
    data = cv2.resize(data, config2.SIZE, interpolation = cv2.INTER_AREA)
    data2 = cv2.resize(data2, config.SIZE, interpolation = cv2.INTER_AREA)
#     cv2.imwrite(SAVE_FOLDER + f"{patient_id}/{img_id}.png", data)
#     cv2.imwrite(SAVE_FOLDER + f"{patient_id}/{img_id}-2.png", data2)
    
#     dummy_images_dict = {}
#     dummy_images_dict[f"{patient_id}/{img_id}.png"] = data
#     dummy_images_dict[f"{patient_id}/{img_id}-2.png"] = data2
#     return dummy_images_dict
    return data, data2

In [10]:
# 1. Since background in a image has constant values in a row- and col-wise direction,
# the breast of interest can be extracted by detecting non-constant elements.
# 2. Even if some objects other than breasts were seen in a image,
# because they are usually smaller, we can extract the breast by picking up the larger one.
# Heuristic way may not be always perfect....

# Count up the continuing "1"
# ex. [0,1,1,0,1,0,0,1,1,1,0] -> [-1,2,2,-1,1,-1,-1,3,3,3,-1]
# Pytroch
def CountUpContinuingOnes_torch(b_arr):
    # indice continuing zeros from left side.
    # [0,1,1,0,1,0,0,1,1,1,0] -> [0,0,0,3,3,5,6,6,6,6,10]
    left = torch.arange(len(b_arr))
    left[b_arr>0] = 0
    left = torch.cummax(left, dim=-1)[0]
    
    # from right side.
    # [0,1,1,0,1,0,0,1,1,1,0] -> [0,3,3,3,5,5,6,10,10,10,10]
    rev_arr = torch.flip(b_arr, [-1])
    right = torch.arange(len(rev_arr))
    right[rev_arr>0] = 0
    right = torch.cummax(right, dim=-1)[0]
    right = len(rev_arr) - 1 - torch.flip(right, [-1])
    
    return right - left - 1

import cupy as cp
# Count up the continuing "1"
# Ex. [0,1,1,0,1,0,0,1,1,1,0] -> [-1,2,2,-1,1,-1,-1,3,3,3,-1]
# Cupy
def CountUpContinuingOnes_cupy(b_arr):
    # indice continuing zeros from left side.
    # ex: [0,1,1,0,1,0,0,1,1,1,0] -> [0,0,0,3,3,5,6,6,6,6,10]
    left = cp.arange(len(b_arr))
    left[b_arr>0] = 0
    left = cupy_cumulative_max(left)
    
    # from right side.
    # ex: [0,1,1,0,1,0,0,1,1,1,0] -> [0,3,3,3,5,5,6,10,10,10,10]
    rev_arr = b_arr[::-1]
    right = cp.arange(len(rev_arr))
    right[rev_arr>0] = 0
    right = cupy_cumulative_max(right)
    right = len(rev_arr) - 1 - right[::-1]
    
    return right - left - 1

import cupy as cp
def cupy_cumulative_max(arr):
    cumulative_max = cp.empty_like(arr)
    current_max = arr[0]
    cumulative_max[0] = current_max
    for i in range(1, len(arr)):
        current_max = cp.maximum(current_max, arr[i])
        cumulative_max[i] = current_max
    return cumulative_max


def ExtractBreast_cupy(img):
    img_copy = img.copy()
    img = cp.where(img <= 40, 0, img) # To detect backgrounds easily
    height, _ = img.shape
    
    # whether each col is non-constant or not
    y_a = height // 2 + int(height*0.4)
    y_b = height // 2 - int(height*0.4)
    b_arr = img[y_b:y_a].std(axis=0) != 0
    continuing_ones = CountUpContinuingOnes_cupy(b_arr)
    # longest should be the breast
    col_ind = cp.where(continuing_ones == continuing_ones.max())[0]
    img = img[:, col_ind]
    
    # whether each row is non-constant or not
    _, width = img.shape
    x_a = width // 2 + int(width*0.4)
    x_b = width // 2 - int(width*0.4)
    b_arr = img[:,x_b:x_a].std(axis=1) != 0
    continuing_ones = CountUpContinuingOnes_cupy(b_arr)
    # longest should be the breast
    row_ind = cp.where(continuing_ones == continuing_ones.max())[0]
    
    return cp.asnumpy(img_copy[row_ind][:, col_ind])


def ExtractBreast_torch(img):
    img_copy = torch.clone(img)
    img = torch.where(img <= 40, torch.zeros_like(img), img) # To detect backgrounds easily
    height, _ = img.shape
    
    # whether each col is non-constant or not
    y_a = height // 2 + int(height*0.4)
    y_b = height // 2 - int(height*0.4)
    b_arr = torch.std(img[y_b:y_a], axis=0) != 0
    continuing_ones = CountUpContinuingOnes_torch(b_arr)
    # longest should be the breast
    col_ind = torch.where(continuing_ones == continuing_ones.max())[0]
    img = img[:, col_ind]
    
    # whether each row is non-constant or not
    _, width = img.shape
    x_a = width // 2 + int(width*0.4)
    x_b = width // 2 - int(width*0.4)
#     b_arr = img[:,x_b:x_a].std(axis=1) != 0
    b_arr = torch.std(img[:, x_b:x_a], axis=1) != 0
    continuing_ones = CountUpContinuingOnes_torch(b_arr)
    # longest should be the breast
    row_ind = torch.where(continuing_ones == continuing_ones.max())[0]
    
    return img_copy[row_ind][:, col_ind]


# Dataset

In [11]:
mean_2 = 0.340530064
std_2 = 0.224438599
class MammoDataset2(Dataset):
    def __init__(self, df, train=True, tfms=None):
        self.df = df
        self.train = train
        self.tfms = tfms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        data = self.df.iloc[idx]
        img = images_dict[f"{data['patient_id']}/{data['image_id']}-2.png"]
        if self.tfms:
            augmented = self.tfms(image=img)
            img = augmented['image']
        img = img.astype('float32')
        img -= img.min()
        img /= img.max()
        img = torch.tensor((img - mean_2)/std_2, dtype=torch.float32)
        if self.train:
            return img.unsqueeze(0), torch.tensor(data['cancer'], dtype=torch.long)
        else:
            return img.unsqueeze(0)

# Model

In [12]:
def gem(x, p:int=3, eps:float=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1.0 / p)


class GeM(nn.Module):
    def __init__(self, p:int=3, eps:float=1e-6, p_trainable=False):
        super(GeM, self).__init__()
        if p_trainable:
            self.p = Parameter(torch.ones(1) * p)
        else:
            self.p = p
        self.eps = eps

    def forward(self, x):
        ret = gem(x, p=self.p, eps=self.eps)
        return ret

    def __repr__(self):
        return (
            self.__class__.__name__
            + "("
            + "p="
            + "{:.4f}".format(self.p.data.tolist()[0])
            + ", "
            + "eps="
            + str(self.eps)
            + ")"
        )
    
class MammoModel(nn.Module):
    def __init__(self, name, *, pretrained=False, in_chans=1, p=3, p_trainable=False, eps=1e-6):
        super().__init__()
        model = timm.create_model(name, pretrained=pretrained, in_chans=in_chans)
        clsf = model.default_cfg['classifier']
        n_features = model._modules[clsf].in_features
        model._modules[clsf] = nn.Identity()
        
        self.fc = nn.Linear(n_features, 1)
        self.model = model

        self.pool = nn.Sequential(
            GeM(p=p, eps=eps, p_trainable=p_trainable),
            nn.Flatten())
    
    def forward(self, x):
        # x = self.model(x)
        x = self.model.forward_features(x)
        x = self.pool(x)
        logits = self.fc(x)
        return logits

In [13]:
def inference_fn(test_loader, model, device):
    model.eval()
    preds = []
    for inputs in test_loader:
        inputs = inputs.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.squeeze().sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions


import torch.cuda.amp as amp
def inference_fn_compile(test_loader, model, device, batch_size):
    model.eval()
    preds = []
    for inputs in test_loader:
#         inputs = inputs.to(device).half()
        inputs = pad_to_batch_size(inputs, batch_size)
        inputs = inputs.to(device)
        with torch.no_grad():
            with amp.autocast(enabled=True):
                y_preds = model(inputs)
        preds.append(y_preds.squeeze().sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions


def inference_fn_compile_tta(test_loader, model, device, batch_size):
    model.eval()
    preds = []
    for inputs in tqdm(test_loader):
#         inputs = inputs.to(device).half()
        inputs = inputs.to(device)
        inputs = pad_to_batch_size(inputs, batch_size=batch_size)
        with torch.no_grad():
            with amp.autocast(enabled=True):
                y_preds = model(inputs).squeeze().sigmoid() / 2
                # vertical flip tta
                y_preds += model(torch.flip(inputs, [-2])).squeeze().sigmoid() / 2
        preds.append(y_preds.to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions


def pad_to_batch_size(image, batch_size):
    B = len(image)
    if B == batch_size:
        return image
    pad = F.pad(input=image, pad=(0, 0, 0, 0, 0, 0, 0, batch_size - B), mode='constant', value=0)
    return pad

In [14]:
%%writefile model_compile.py
# https://www.kaggle.com/code/tivfrvqhs5/torch-tensorrt-infer-fp16-and-fp32-benchmarks/notebook
# https://www.kaggle.com/code/hengck23/3hr-tensorrt-nextvit-example/notebook
# https://www.kaggle.com/competitions/rsna-breast-cancer-detection/discussion/375881

import sys
sys.path.append('/kaggle/input/timm-0-6-12-tf-effv2s-1520-912')

import gdcm
import glob

import os
import pickle
import random
import math

from joblib import Parallel, delayed
import shutil
import cv2
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import timm
from albumentations  import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pydicom.pixel_data_handlers import apply_windowing
import dicomsdl
import gc
import torch.backends.cudnn as cudnn

import matplotlib.pyplot as plt

print('torch version:', torch.__version__)
print('timm version:', timm.__version__)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device:', device)

import torch
import torch.nn.functional as F
import nvidia.dali.fn as fn
import nvidia.dali.types as types
from nvidia.dali import pipeline_def
from nvidia.dali.types import DALIDataType
from pydicom.filebase import DicomBytesIO
from nvidia.dali.plugin.pytorch import feed_ndarray, to_torch_type


os.environ['CUDA_MODULE_LOADING']='LAZY'

import torch_tensorrt
import tensorrt
cudnn.benchmark = True

class config:
    seed = 10
    batch_size = 16
    n_folds = 5
    name = 'tf_efficientnetv2_s'
    SIZE = (1024, 1024)  
class config2:
    seed = 10
    batch_size = 16
    n_folds = 4
    name = 'tf_efficientnetv2_s'
    SIZE = (912, 1520)
    
    
def seed_everything(seed: int):    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(config.seed)

def gem(x, p:int=3, eps:float=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1.0 / p)


class GeM(nn.Module):
    def __init__(self, p:int=3, eps:float=1e-6, p_trainable=False):
        super(GeM, self).__init__()
        if p_trainable:
            self.p = Parameter(torch.ones(1) * p)
        else:
            self.p = p
        self.eps = eps

    def forward(self, x):
        ret = gem(x, p=self.p, eps=self.eps)
        return ret

    def __repr__(self):
        return (
            self.__class__.__name__
            + "("
            + "p="
            + "{:.4f}".format(self.p.data.tolist()[0])
            + ", "
            + "eps="
            + str(self.eps)
            + ")"
        )
    
class MammoModel(nn.Module):
    def __init__(self, name, *, pretrained=False, in_chans=1, p=3, p_trainable=False, eps=1e-6):
        super().__init__()
        model = timm.create_model(name, pretrained=pretrained, in_chans=in_chans)
        clsf = model.default_cfg['classifier']
        n_features = model._modules[clsf].in_features
        model._modules[clsf] = nn.Identity()
        
        self.fc = nn.Linear(n_features, 1)
        self.model = model

        self.pool = nn.Sequential(
            GeM(p=p, eps=eps, p_trainable=p_trainable),
            nn.Flatten())
    
    def forward(self, x):
        # x = self.model(x)
        x = self.model.forward_features(x)
        x = self.pool(x)
        logits = self.fc(x)
        return logits
    
def tensorrt_compile_v2s(path, config, name, fold):
    model = MammoModel(config.name, pretrained=False)
    current_params = model.state_dict()
    trained_params = torch.load(path, map_location='cpu')['model']
    for key in current_params.keys():
        if key == 'model.conv_stem.conv.weight':
            current_params[key] = trained_params['model.conv_stem.weight']
        elif key == 'model.blocks.1.0.conv_exp.conv.weight':
            current_params[key] = trained_params['model.blocks.1.0.conv_exp.weight']
        elif key == 'model.blocks.2.0.conv_exp.conv.weight':
            current_params[key] = trained_params['model.blocks.2.0.conv_exp.weight']
        elif key == 'model.blocks.3.0.conv_dw.conv.weight':
            current_params[key] = trained_params['model.blocks.3.0.conv_dw.weight']
        else:
            current_params[key] = trained_params[key]
    model.load_state_dict(current_params)
#     model.half()  ### should not be used !!!!!
    model.eval().to(device)
    # model.encoder.merge_bn()

    # The compiled module will have precision as specified by "op_precision".
    # with torch_tensorrt.logging.debug():
    trt_model_fp16 = torch_tensorrt.compile(
        model,
        inputs=[
            torch_tensorrt.Input(
            [config.batch_size*2, 1, config.SIZE[1], config.SIZE[0]],
            dtype=torch.float32
        )],
        enabled_precisions={torch.float32},  # Run with FP16
        workspace_size=1 << 32,
    #     debug=True,
        require_full_compilation=True,
    ) 
    torch.jit.save(trt_model_fp16, f'{name}-{fold}.ts')
    
    del trt_model_fp16
    torch.cuda.empty_cache()
    gc.collect()
    print('ok')
    
    
def tensorrt_compile_B5(path, config, name, fold):
    model = MammoModel(config.name, pretrained=False)
    current_params = model.state_dict()
    trained_params = torch.load(path, map_location='cpu')['model']
    for key in current_params.keys():
        if key == 'model.conv_stem.conv.weight':
            current_params[key] = trained_params['model.conv_stem.weight']
        elif key == 'model.blocks.1.0.conv_dw.conv.weight':
            current_params[key] = trained_params['model.blocks.1.0.conv_dw.weight']
        elif key == 'model.blocks.2.0.conv_dw.conv.weight':
            current_params[key] = trained_params['model.blocks.2.0.conv_dw.weight']
        elif key == 'model.blocks.3.0.conv_dw.conv.weight':
            current_params[key] = trained_params['model.blocks.3.0.conv_dw.weight']
        else:
            current_params[key] = trained_params[key]
    model.load_state_dict(current_params)
#     model.half()  ### should not be used !!!!!
    model.eval().to(device)
    # model.encoder.merge_bn()

    # The compiled module will have precision as specified by "op_precision".
    # with torch_tensorrt.logging.debug():
    trt_model_fp16 = torch_tensorrt.compile(
        model,
        inputs=[
            torch_tensorrt.Input(
            [config.batch_size*2, 1, config.SIZE[1], config.SIZE[0]],
            dtype=torch.float32
        )],
        enabled_precisions={torch.float32},  # Run with FP16
        workspace_size=1 << 32,
    #     debug=True,
        require_full_compilation=True,
    ) 
    torch.jit.save(trt_model_fp16, f'{name}-{fold}.ts')
    
    del trt_model_fp16
    torch.cuda.empty_cache()
    gc.collect()
    print('ok')


for fold in range(config.n_folds):
    print('fold:', fold)
    path = f'/kaggle/input/ver49-weight3/efficientnetv2_s_seed_10_fold{fold}_best_ver49_swa.pth'
    name = 'ver49'
    tensorrt_compile_v2s(path, config, name, fold)
    
for fold in range(config2.n_folds):
    print('fold:', fold)
    path = f'/kaggle/input/rsna-efficientnetv2-s-pl-4folds/efficientnetv2_s_pl_seed_10_fold{fold}_best_score_ver084.pth'
    name = 'ver84'
    tensorrt_compile_v2s(path, config2, name, fold)
    
for fold in range(config3.n_folds):
    print('fold:', fold)
    path = f'/kaggle/input/rsna-efficientnetb5-pl-4folds/efficientnetb5_pl_seed_10_fold{fold}_best_score_ver083.pth'
    name = 'ver83'
    tensorrt_compile_B5(path, config3, name, fold)

print('trt_fp16 ok')

Writing model_compile.py


In [15]:
gc.collect()
if use_compile:
    !python model_compile.py
else:
    !rm model_compile.py

In [16]:
phase = 'test' # 'test'

if phase == 'train':
    df = train_df[:5000]
else:
    if len(test_df) < 100:
        phase = 'train'
        df = train_df[:100]
    else:
        df = test_df
    

IMG_PATH = f"/kaggle/input/rsna-breast-cancer-detection/{phase}_images/"
# test_images = glob.glob(f"{IMG_PATH}*/*.dcm")
    
print("Number of images :", len(df))

SAVE_FOLDER = f'{phase}_images/'


os.makedirs(SAVE_FOLDER, exist_ok=True)
for patient_id in df['patient_id'].unique():
    os.makedirs(SAVE_FOLDER+str(patient_id), exist_ok=True)

if len(df) > 10000:
    N_CHUNKS = 100
else:
    N_CHUNKS = 1

CHUNKS = [(len(df) / N_CHUNKS * k, len(df) / N_CHUNKS * (k + 1)) for k in range(N_CHUNKS)]
CHUNKS = np.array(CHUNKS).astype(int)

# J2K_FOLDER = "/tmp/j2k/"
JPG_FOLDER = "/tmp/jpg/"

if len(df) <= 10000:
    debug_flag = True
else:
    debug_flag = False

if debug_flag:
    config.n_folds = 2

Number of images : 100


## Generate images

In [17]:
csize = len(df['patient_id'].unique())//N_CHUNKS
patients = []
for i in range(0, len(df['patient_id'].unique()), csize):
    patients.append(df['patient_id'].unique()[i:i+csize])
    
CHUNKS = []
for p in patients:
    CHUNKS.append([df[df.patient_id.isin(p)].index[0], df[df.patient_id.isin(p)].index[-1]+1])

In [18]:
df_all = df.copy()
for chunk in tqdm(CHUNKS):
    print(chunk)

  0%|          | 0/1 [00:00<?, ?it/s]

[0, 100]


In [19]:
def save_png(patient_id, img_id):
    cv2.imwrite(SAVE_FOLDER + f"{patient_id}/{img_id}.png", images_dict[f"{patient_id}/{img_id}.png"])
df_all['pat_lat'] = df_all['patient_id'].astype(str) + df_all['laterality']
gc.collect()

0

In [20]:
test_preds_array = np.array([])
second_df = pd.DataFrame()
if torch.cuda.is_available():
# if True:

    for chunk in tqdm(CHUNKS):
        print('start')
        df = df_all.iloc[chunk[0]: chunk[1]]
        images_dict = {}
        
        os.makedirs(JPG_FOLDER, exist_ok=True)


        _ = Parallel(n_jobs=2)(
            delayed(convert_dicom_to_jpg)(patient_id, img_id, save_folder=JPG_FOLDER)
            for patient_id, img_id in zip(df['patient_id'].values, df['image_id'].values)
        )
    
        jpgfiles = glob.glob(JPG_FOLDER + "*.jpg")

        if len(jpgfiles):
            pipe = jpg_decode_pipeline(jpgfiles, batch_size=1, num_threads=2, device_id=0)
            pipe.build()

        for f in jpgfiles:
            patient, image = f.split('/')[-1][:-4].split('_')
            dicom = pydicom.dcmread(IMG_PATH + f"{patient}/{image}.dcm")
            try:

                out = pipe.run()

                # Dali -> Torch
                img = out[0][0]
                img_torch = torch.empty(img.shape(), dtype=torch.int16, device="cuda")
                feed_ndarray(img, img_torch, cuda_stream=torch.cuda.current_stream(device=0))
                img = img_torch.float()
                img = img.reshape(img.shape[0], img.shape[1])


                # read_mammography()に相当
                img = img[5:-5, 5:-5]
                
                rev = 0
                rev_val = img.max().item() # for sigmoid windowing
                if dicom.PhotometricInterpretation == "MONOCHROME1":
                    img = img.max() - img
                min_, max_ = img.min(), img.max()
                img = (img - min_) / (max_ - min_)
                img = (img * 255)
                
                if dicom['WindowWidth'].VM > 1:
                    center = float(dicom.WindowCenter[0])
                    width = float(dicom.WindowWidth[0])
                else:
                    center = float(dicom.WindowCenter)
                    width = float(dicom.WindowWidth)
                y_range = 2**dicom.BitsStored - 1
                windowing_stat_dict[f'{image}'] = [min_.item(), max_.item(), center, width, y_range, rev, rev_val]

                img = ExtractBreast_torch(img).cpu().numpy().astype(np.uint8)

                c_h = img.shape[0]
                if c_h > 2048:
                    pad = c_h - 2048
                    img2 = img[pad//2:-pad//2, :]

                img = cv2.resize(img, config2.SIZE, interpolation =cv2.INTER_AREA)
                img2 = cv2.resize(img2, config.SIZE, interpolation =cv2.INTER_AREA)

    #             fig, axes = plt.subplots(figsize=(5, 5), ncols=1, nrows=1)
    #             axes.imshow(img, cmap='bone')

                images_dict[f"{patient}/{image}.png"] = img
                images_dict[f"{patient}/{image}-2.png"] = img2
            
            except Exception as e:
                print(patient, image)

        shutil.rmtree(JPG_FOLDER)
        torch.cuda.empty_cache()
        gc.collect()
        
        
        for patient_id, img_id in tqdm(zip(df['patient_id'].values, df['image_id'].values)):
            images = save_array(patient_id, img_id, images_dict)
            if images:
                images_dict[f"{patient_id}/{img_id}.png"] = images[0] # 912*1520
                images_dict[f"{patient_id}/{img_id}-2.png"] = images[1] # 1024*1024
    
        
        test_preds = np.zeros(len(df), dtype='float64')
    
    
        test_dataset = MammoDataset2(df, train=False)
        test_loader = DataLoader(test_dataset,
                                     batch_size=config.batch_size,
                                     shuffle=False,
                                     num_workers=2, pin_memory=True, drop_last=False)
        
        for fold in range(config.n_folds):
            model = torch.jit.load(f'/kaggle/input/compile-ver49-83-83/ver49-{fold}.ts')
            pred = inference_fn_compile(test_loader, model, device, config.batch_size)
            pred = pred[:chunk[1] - chunk[0]]
            test_preds += pred/config.n_folds

#         test_preds = np.random.rand(len(df))
        df['prediction'] = test_preds
        df_agg = df[['patient_id', 'laterality', 'prediction', 'pat_lat']].groupby(['patient_id', 'laterality', 'pat_lat']).mean()
        df_agg = df_agg[(df_agg['prediction'] > 0.01) & (df_agg['prediction'] < 0.9)].reset_index()
        
        df = df[df.pat_lat.isin(set(df_agg['pat_lat']))]
        
        del model
        torch.cuda.empty_cache()
        gc.collect()
        
#         _ = Parallel(n_jobs=2)(
#             delayed(save_png)(patient_id, img_id)
#             for patient_id, img_id in zip(df['patient_id'].values, df['image_id'].values)
#         )
        for patient_id, img_id in tqdm(zip(df['patient_id'].values, df['image_id'].values)):
            save_png(patient_id, img_id)
        
        second_df = pd.concat([second_df, df])
            
        test_preds_array = np.concatenate([test_preds_array, test_preds])
        
        del images_dict
        torch.cuda.empty_cache()
        gc.collect()
        print('end')
second_df = second_df.reset_index(drop=True)

  0%|          | 0/1 [00:00<?, ?it/s]

start


0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0it [00:00, ?it/s]

end


In [21]:
second_df

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,pat_lat,prediction
0,1,10049,94335194,R,MLO,52.0,0,0,0,0.0,0,C,49,True,10049R,0.006018
1,1,10049,1207499426,R,MLO,52.0,0,0,0,0.0,0,C,49,True,10049R,0.011007
2,1,10049,1351270472,R,CC,52.0,0,0,0,0.0,0,C,49,True,10049R,0.051407
3,2,10050,588678397,L,MLO,67.0,0,0,0,NaN,0,NaN,29,False,10050L,0.085673
4,2,10050,1749389520,L,CC,67.0,0,0,0,NaN,0,NaN,29,False,10050L,0.012839
5,2,10050,1428987847,R,MLO,67.0,0,0,0,NaN,0,NaN,29,False,10050R,0.016540
6,2,10050,1614607569,R,CC,67.0,0,0,0,NaN,0,NaN,29,False,10050R,0.009267
7,2,10116,472095321,L,CC,67.0,0,0,0,NaN,0,NaN,29,False,10116L,0.005207
8,2,10116,539757861,L,MLO,67.0,0,0,0,NaN,0,NaN,29,False,10116L,0.030961
9,2,10116,527043166,R,CC,67.0,0,0,0,NaN,0,NaN,29,False,10116R,0.048905


In [22]:
df = df_all.copy()
df['prediction_ver49'] = test_preds_array
# df['prediction_ver49'] = np.random.rand(len(df))

# Second Stage

# Breast Level

In [23]:
class config:
    seed = 10
    batch_size = 16
    n_folds = 4
    name = 'tf_efficientnetv2_s'
    
if debug_flag:
    config.n_folds = 2

mean = 0.3089279
std = 0.25053555408335154
class MammoDatasetBreast(Dataset):
    def __init__(self, df, train=True, tfms=None):
        self.df = df
        self.train = train
        self.tfms = tfms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        data = self.df.iloc[idx]
        path = SAVE_FOLDER + f"{data['patient_id']}/{data['image_id']}.png"
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if self.tfms:
            augmented = self.tfms(image=img)
            img = augmented['image']
        img = img.astype('float32')
        img -= img.min()
        img /= img.max()
        img = torch.tensor((img - mean)/std, dtype=torch.float32)
        if self.train:
            return img.unsqueeze(0), torch.tensor(data['cancer'], dtype=torch.long)
        else:
            return img.unsqueeze(0)
        
        
mean = 0.3089279
std = 0.25053555408335154
class MammoDatasetPatLat(Dataset):
    def __init__(self, df, train=True, tfms=None, size=1520):
        self.df = df
        self.train = train
        self.tfms = tfms
        self.size = (size, size)
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        data = self.df.iloc[idx]
        imgs = []
        for img_id in data['image_id']:
            path = SAVE_FOLDER + f"{data['patient_id']}/{img_id}.png"
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            imgs.append(img)
        imgs = np.concatenate(imgs, axis=1)
        imgs = cv2.resize(imgs, self.size, interpolation=cv2.INTER_AREA)
        
#         path = BASE_PATH + f"{data['patient_id']}/" + str(data['image_id']) + '.png'
#         path = f"test_images/{data['patient_id']}/{data['image_id']}.png"
        # img = pickle.load(open(path, 'rb'))
#         img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if self.tfms:
            augmented = self.tfms(image=imgs)
            imgs = augmented['image']
#         img = torch.tensor((img/255. - mean)/std, dtype=torch.float32)
        imgs = imgs.astype('float32')
        imgs -= imgs.min()
        imgs /= imgs.max()
        imgs = torch.tensor((imgs - mean)/std, dtype=torch.float32)
        if self.train:
            return imgs.unsqueeze(0), torch.tensor(data['cancer'], dtype=torch.long)
        else:
            return imgs.unsqueeze(0)

# sigmoid windowing; sw
mean_sw = 0.21902776
std_sw = 0.2058490367876737
class MammoDatasetBreastSW(Dataset):
    def __init__(self, df, train=True, tfms=None):
        self.df = df
        self.train = train
        self.tfms = tfms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        data = self.df.iloc[idx]
        windowing_stat = windowing_stat_dict[f"{data['image_id']}"]
        min_, max_, center_, width_, y_range_, rev_, rev_max_ = windowing_stat
#         path = BASE_PATH + f"{data['patient_id']}/" + str(data['image_id']) + '.png'
#         path = f"test_images/{data['patient_id']}/{data['image_id']}.png"
        path = SAVE_FOLDER + f"{data['patient_id']}/{data['image_id']}.png"
        # img = pickle.load(open(path, 'rb'))
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if self.tfms:
            augmented = self.tfms(image=img)
            img = augmented['image']
#         img = torch.tensor((img/255. - mean)/std, dtype=torch.float32)
        img = img.astype('float32')
        
        # sigmoid windowing
        img /= 255
        img = img * (max_ - min_) + min_
        if rev_ == 1:
            img = rev_max_ - img
        img = y_range_ / (1 + np.exp(-4 * (img - center_) / width_))
        if rev_ == 1:
            img = np.amax(img) - img
        img -= img.min()
        img /= img.max()
        img = torch.tensor((img - mean_sw)/std_sw, dtype=torch.float32)
        if self.train:
            return img.unsqueeze(0), torch.tensor(data['cancer'], dtype=torch.long)
        else:
            return img.unsqueeze(0)

# sigmoid windowing; sw
mean_sw = 0.21902776
std_sw = 0.2058490367876737
class MammoDatasetPatLatSW(Dataset):
    def __init__(self, df, train=True, tfms=None, size=1520):
        self.df = df
        self.train = train
        self.tfms = tfms
        self.size = (size, size)
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        data = self.df.iloc[idx]
        imgs = []
        for n, img_id in enumerate(data['image_id']):
            if n == 0:
                windowing_stat = windowing_stat_dict[f"{img_id}"]
#             path = f"test_images/{data['patient_id']}/{img_id}.png"
            path = SAVE_FOLDER + f"{data['patient_id']}/{img_id}.png"
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            imgs.append(img)
        min_, max_, center_, width_, y_range_, rev_, rev_max_ = windowing_stat
        imgs = np.concatenate(imgs, axis=1)
        
        # sigmoid windowing
        imgs = imgs.astype('float32')
        imgs /= 255
        imgs = imgs * (max_ - min_) + min_
        if rev_ == 1:
            imgs = rev_max_ - imgs
        imgs = y_range_ / (1 + np.exp(-4 * (imgs - center_) / width_))
        if rev_ == 1:
            imgs = np.amax(imgs) - imgs
        imgs -= imgs.min()
        imgs /= imgs.max()
        imgs *= 255
        
        imgs = cv2.resize(imgs, self.size, interpolation=cv2.INTER_AREA)
        
        imgs = imgs.astype('float32')
        imgs -= imgs.min()
        imgs /= imgs.max()
        imgs = torch.tensor((imgs - mean_sw)/std_sw, dtype=torch.float32)
        if self.train:
            return imgs.unsqueeze(0), torch.tensor(data['cancer'], dtype=torch.long)
        else:
            return imgs.unsqueeze(0)
        
test_dataset = MammoDatasetBreast(second_df, train=False)
test_loader = DataLoader(test_dataset,
                             batch_size=32,
                             shuffle=False,
                             num_workers=2, pin_memory=True, drop_last=False)
test_preds_1 = np.zeros(len(second_df), dtype='float64')
for fold in range(config.n_folds):
    model = torch.jit.load(f'/kaggle/input/efficientnetv2s-exp084-tensorrt/efficientnetv2s_ver084-seed10-{fold}-swa.ts')
    pred = inference_fn_compile_tta(test_loader, model, device, 32)
    test_preds_1 += pred[:len(second_df)] / config.n_folds
    
    if fold == 1:
        del model
        torch.cuda.empty_cache()
        gc.collect()
if fold != 1:
    del model
    torch.cuda.empty_cache()
    gc.collect()
    
    
test_preds_2 = np.zeros(len(second_df), dtype='float64')
for fold in range(config.n_folds):
    model = torch.jit.load(f'/kaggle/input/efficientnetv2s-exp084-tensorrt/efficientnetv2s_ver084-seed660-{fold}-swa.ts')
    pred = inference_fn_compile_tta(test_loader, model, device, 32)
    test_preds_2 += pred[:len(second_df)] / config.n_folds
#     second_df[f'prediction{fold}'] = pred

    if fold == 1:
        del model
        torch.cuda.empty_cache()
        gc.collect()
if fold != 1:
    del model
    torch.cuda.empty_cache()
    gc.collect()

# Sigmoid Windowing
test_dataset = MammoDatasetBreastSW(second_df, train=False)
test_loader = DataLoader(test_dataset,
                             batch_size=32,
                             shuffle=False,
                             num_workers=2, pin_memory=True, drop_last=False)
test_preds_3 = np.zeros(len(second_df), dtype='float64')
for fold in range(config.n_folds):
    model = torch.jit.load(f'/kaggle/input/rsna-efficientnetv2s-pseudo-sig-tensorrt/efficientnetv2s_ver099-seed10-{fold}.ts')
    pred = inference_fn_compile(test_loader, model, device, 32)
    test_preds_3 += pred[:len(second_df)] / config.n_folds
#     second_df[f'prediction{fold}'] = pred

    if fold == 1:
        del model
        torch.cuda.empty_cache()
        gc.collect()
if fold != 1:
    del model
    torch.cuda.empty_cache()
    gc.collect()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times



  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multipro

In [24]:
# second_df['prediction_ver84'] = np.random.rand(len(second_df))
# p = 0.45
# test_preds = p * test_preds + (1 - p) * test_preds_2
# second_df['prediction_ver84'] = test_preds
# second_df

In [25]:
p1 = 0.55
p2 = 0.35
test_preds = p1 * test_preds_1 + p2 * test_preds_2 + (1 - p1 - p2) * test_preds_3
second_df['prediction_ver84'] = test_preds
second_df

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,pat_lat,prediction,prediction_ver84
0,1,10049,94335194,R,MLO,52.0,0,0,0,0.0,0,C,49,True,10049R,0.006018,0.016809
1,1,10049,1207499426,R,MLO,52.0,0,0,0,0.0,0,C,49,True,10049R,0.011007,0.022877
2,1,10049,1351270472,R,CC,52.0,0,0,0,0.0,0,C,49,True,10049R,0.051407,0.007190
3,2,10050,588678397,L,MLO,67.0,0,0,0,NaN,0,NaN,29,False,10050L,0.085673,0.097196
4,2,10050,1749389520,L,CC,67.0,0,0,0,NaN,0,NaN,29,False,10050L,0.012839,0.017356
5,2,10050,1428987847,R,MLO,67.0,0,0,0,NaN,0,NaN,29,False,10050R,0.016540,0.006614
6,2,10050,1614607569,R,CC,67.0,0,0,0,NaN,0,NaN,29,False,10050R,0.009267,0.002774
7,2,10116,472095321,L,CC,67.0,0,0,0,NaN,0,NaN,29,False,10116L,0.005207,0.002646
8,2,10116,539757861,L,MLO,67.0,0,0,0,NaN,0,NaN,29,False,10116L,0.030961,0.064222
9,2,10116,527043166,R,CC,67.0,0,0,0,NaN,0,NaN,29,False,10116R,0.048905,0.010794


In [26]:
df = df.merge(second_df, how='outer')
df['prediction_ver84'] = df['prediction_ver84'].fillna(0)
df

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,pat_lat,prediction_ver49,prediction,prediction_ver84
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False,10006L,0.001729,NaN,0.0
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False,10006L,0.001527,NaN,0.0
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False,10006R,0.005830,NaN,0.0
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False,10006R,0.004521,NaN,0.0
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True,10011L,0.000213,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,10132,1646384157,L,CC,58.0,0,1,0,0.0,0,B,49,True,10132L,0.009777,NaN,0.0
96,1,10132,441108517,R,MLO,58.0,0,0,0,NaN,0,B,49,False,10132R,0.001892,NaN,0.0
97,1,10132,2137344394,R,CC,58.0,0,0,0,NaN,0,B,49,False,10132R,0.000933,NaN,0.0
98,2,10136,912156277,L,CC,46.0,0,0,0,0.0,0,NaN,48,True,10136L,0.000849,NaN,0.0


In [27]:
df = df[['patient_id', 'laterality', 'prediction_ver49', 'prediction_ver84']].groupby(['patient_id', 'laterality']).mean()
df = df.reset_index()
df['prediction_id'] = [f'{p}_{l}' for p, l in zip(df['patient_id'].values, df['laterality'].values)]
df.index = df['prediction_id'].values

# Patient-Laterality Level

In [28]:
test_df_patlat = second_df.copy()
test_df_patlat = test_df_patlat.loc[[v in ['CC', 'MLO'] for v in test_df_patlat['view'].values]].reset_index(drop=True) # exclude minor view images
image_ids = test_df_patlat.groupby(['patient_id', 'laterality'])['image_id'].apply(list).reset_index()['image_id'].values
test_df_patlat = test_df_patlat.groupby(['patient_id', 'laterality']).first().reset_index()
test_df_patlat['image_id'] = image_ids

In [29]:
test_dataset = MammoDatasetPatLat(test_df_patlat, train=False)
test_loader = DataLoader(test_dataset,
                             batch_size=16,
                             shuffle=False,
                             num_workers=2, pin_memory=True, drop_last=False)
test_preds_1 = np.zeros(len(test_df_patlat), dtype='float64')
for fold in range(config.n_folds):
    model = torch.jit.load(f'/kaggle/input/efficientnetv2s-exp085-tensorrt/efficientnetv2s_ver085-{fold}.ts')
    pred = inference_fn_compile(test_loader, model, device, 16)
    test_preds_1 += pred[:len(test_df_patlat)] / config.n_folds
    if fold == 1:
        del model
        torch.cuda.empty_cache()
        gc.collect()
if fold != 1:
    del model
    torch.cuda.empty_cache()
    gc.collect()
    

test_preds_2 = np.zeros(len(test_df_patlat), dtype='float64')
for fold in range(config.n_folds):
    model = torch.jit.load(f'/kaggle/input/efficientnetv2s-exp085-tensorrt/efficientnetv2s_ver085-seed660-{fold}.ts')
    pred = inference_fn_compile(test_loader, model, device, 16)
    test_preds_2 += pred[:len(test_df_patlat)] / config.n_folds
    
    if fold == 1:
        del model
        torch.cuda.empty_cache()
        gc.collect()
if fold != 1:
    del model
    torch.cuda.empty_cache()
    gc.collect()

test_dataset = MammoDatasetPatLatSW(test_df_patlat, train=False)
test_loader = DataLoader(test_dataset,
                             batch_size=16,
                             shuffle=False,
                             num_workers=2, pin_memory=True, drop_last=False)
test_preds_3 = np.zeros(len(test_df_patlat), dtype='float64')
for fold in range(config.n_folds):
    model = torch.jit.load(f'/kaggle/input/rsna-efficientnetv2s-pseudo-sig-tensorrt/efficientnetv2s_ver100-seed10-{fold}.ts')
    pred = inference_fn_compile(test_loader, model, device, 16)
    test_preds_3 += pred[:len(test_df_patlat)] / config.n_folds
    
    if fold == 1:
        del model
        torch.cuda.empty_cache()
        gc.collect()
if fold != 1:
    del model
    torch.cuda.empty_cache()
    gc.collect()

In [30]:
# p = 0.4
# test_preds = p * test_preds + (1 - p) * test_preds_2
# test_df_patlat['prediction_ver85'] = test_preds
# test_df_patlat['prediction_ver85'] = np.random.rand(len(test_df_patlat))

# if not 'prediction_id' in test_df_patlat.columns:
#     test_df_patlat['prediction_id'] = [f'{p}_{l}' for p, l in zip(test_df_patlat['patient_id'].values, test_df_patlat['laterality'].values)]
# df = df.merge(test_df_patlat[['prediction_id', 'prediction_ver85']], how='outer')
# df['prediction_ver85'] = df['prediction_ver85'].fillna(0)
# df

In [31]:
p1 = 0.45
p2 = 0.2
test_preds = p1 * test_preds_1 + p2 * test_preds_2 + (1 - p1 - p2) * test_preds_3
test_df_patlat['prediction_ver85'] = test_preds

if not 'prediction_id' in test_df_patlat.columns:
    test_df_patlat['prediction_id'] = [f'{p}_{l}' for p, l in zip(test_df_patlat['patient_id'].values, test_df_patlat['laterality'].values)]
df = df.merge(test_df_patlat[['prediction_id', 'prediction_ver85']], how='outer')
df['prediction_ver85'] = df['prediction_ver85'].fillna(0)
df

,patient_id,laterality,prediction_ver49,prediction_ver84,prediction_id,prediction_ver85
0,10006,L,0.001628,0.000000,10006_L,0.000000
1,10006,R,0.005175,0.000000,10006_R,0.000000
2,10011,L,0.000291,0.000000,10011_L,0.000000
3,10011,R,0.000401,0.000000,10011_R,0.000000
4,10025,L,0.004324,0.000000,10025_L,0.000000
5,10025,R,0.006225,0.000000,10025_R,0.000000
6,10038,L,0.001244,0.000000,10038_L,0.000000
7,10038,R,0.000616,0.000000,10038_R,0.000000
8,10042,L,0.000495,0.000000,10042_L,0.000000
9,10042,R,0.000687,0.000000,10042_R,0.000000


In [32]:
# weight1 = 0.54
# weight2 = 0.81
# th = 0.185
p = 0.75
th = 0.225
# test_preds = weight2 * (weight1 * df['prediction_ver49'].values + (1 - weight1) * df['prediction_ver84'].values) + (1 - weight2) * df['prediction_ver85'].values
test_preds = p*df['prediction_ver84'].values + (1-p)*df['prediction_ver85'].values
df['prediction'] = test_preds
df['max'] = [max(i, j, k) for i,j,k in zip(df['prediction_ver49'], df['prediction_ver84'], df['prediction_ver85'])]
df['prediction'] = [j if j > 0.9 else i for i, j in zip(df['prediction'],df['max'])]
test_preds = df['prediction'].values
test_preds = (test_preds >= th).astype('int')

In [33]:
submission = pd.DataFrame({'prediction_id': df['prediction_id'].values,
                         'cancer': test_preds})
submission.to_csv('submission.csv', index=False)

In [34]:
submission.head()

,prediction_id,cancer
0,10006_L,0
1,10006_R,0
2,10011_L,0
3,10011_R,0
4,10025_L,0
